In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from datetime import datetime as dt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import re, string, sys, pprint
from datetime import datetime

sns.set()
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#========================================================
#submission line with output table in submission format, this file will show up in output folder on right
# output.to_csv('submission.csv,index=False')

See, fork, and run a random forest benchmark model through Kaggle Scripts

You are provided hourly rental data spanning two years 1/1/2011-31/12/2012. For this competition, the training set is comprised of the first 19 days of each month, while the test set is the 20th to the end of the month. You must predict the total count of bikes rented during each hour covered by the test set, using only information available prior to the rental period.



Data Fields
datetime - hourly date + timestamp  
season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
holiday - whether the day is considered a holiday
workingday - whether the day is neither a weekend nor holiday
weather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
temp - temperature in Celsius
atemp - "feels like" temperature in Celsius
humidity - relative humidity
windspeed - wind speed
casual - number of non-registered user rentals initiated
registered - number of registered user rentals initiated
count - number of total rentals

In [ ]:
trainset = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv', index_col=False)
testset = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv', index_col=False)

train=trainset.copy()
test=testset.copy()
display(train.head(3))
display(test.head(3))

In [ ]:
def exploreset(mydataset):
    print("\n=============\n")
    print("COLUMNS IN DATASET: ","\n", mydataset.columns,"\n")
    print("ROWS/INDEX IN DATASET: ","\n", mydataset.index,"\n")
    print('SHAPE OF DATASET: ',"\n",mydataset.shape,"\n")
    print('SIZE OF DATASET: ',"\n",mydataset.size)
    print("\n=============\n")
    print('COUNT of ISNA in each COLUMN',"\n-----------")
    print(mydataset.isna().sum(axis=0))
    print("\n=============\n")
    print('DATASET INFO',"\n-----------")
    print(mydataset.info(),"\n")
    print("\n=============\n")
    print('DATASET DESCRIPTION',"\n-----------")
    print(mydataset.describe(),"\n")
    print("\n=============\n")
    print('DATASET CORRELATIONS',"\n-----------")
    print(mydataset.corr(),"\n")

        
def ISNAValues(mydataset):
    print('COLUMNS with ISNA VALUES',"\n===")
    for column_name, isna in mydataset.isna().items():
        try:
            if isna.sum() > 0:

                    print("//", column_name, ' type     : ', type(mydataset[column_name][1]))
                    print("//", column_name, ' na count : ', isna.sum(),"\n")

        except:
            print("Error","\n")


def uniquevalues(mydataset):
    print("\n=============\n")
   
    print('COUNT of UNIQUE VALUES in each COLUMN',"\n-----------")
    print(mydataset.nunique(),"\n")
 
    print('DETAILS OF COLUMNS WITH UNIQUE VALUES >0',"\n===")
    for column_name, nunique in mydataset.nunique().items():
        if nunique >0:
            print(column_name, ':', mydataset[column_name].unique(),'// uniq : ', round(mydataset[column_name].nunique(),1))
            print(mydataset[column_name].value_counts(),"\n====\n")
            
 

In [ ]:
def run_analysis_imports(mydataset):
    exploreset(mydataset)
    ISNAValues(mydataset)
    uniquevalues(mydataset)   

In [ ]:
print('TRAIN DATA')
run_analysis_imports(train)

## Train set data

no na values 

Count of unique values show

datetime      10886 unique datetime values, (hourly date + timestamp) is an object and needs processing
season            4 categorical data where season - 1 = spring, 2 = summer, 3 = fall, 4 = winter holiday
holiday           2 categorical data where 1 = yes/0 = no
workingday        2 categorical data where 1 = yes/0 = no (whether the day is neither a weekend nor holiday ie. may overlap)

weather           4  categorical data where (1, 2, 3, 4)
    1: Clear, Few clouds, Partly cloudy, Partly cloudy 
    2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist 
    3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds 
    4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
temp             49 continuous, temperature in Celsius
atemp            60 continuous, "feels like" temperature in Celsius 
humidity         89 discrete, relative humidity
windspeed        28 continuous, wind speed 

casual          309 discrete, number of non-registered user rentals initiated
registered      731 discrete, number of registered user rentals initiated
count           822 discrete, number of total rentals (ie casual + registered) (RESPONSE)


DATASET DESCRIPTION (2 yrs of hourly data for 20 days/mth)

Weather hourly mean is 1.4 indicating that the rental coverage area is one that tends to have clear or misty weather more often than snow, or thunderstorms

Hourly Temp ranges from approx. 0 to 45 degree celcius with the mean temperature around 20 deg celcius

Mean hourly rental by registered users is 155 much higher than mean hourly rental by casual users 36




In [ ]:
print('TEST DATA')
run_analysis_imports(test)


# columns in train but not in test data >>  
# casual - number of non-registered user rentals initiated
# registered - number of registered user rentals initiated
# count - number of total rentals

#count of each date in test data need to be predicted

##Test set data

no na values
Matching columns and data types to train set data except for casual, registered, count are not available


In [ ]:
# Feature enhancement of date/time 2011-01-20 00:00:00 

display(test.sample(5))
display(train.sample(5))

### Train, Test Data setup + Adding New Features 

In [ ]:
# Categorical 'am' column 

def ampm(row):
    am_list = list(range(12))
    if row['hour'] in am_list:
        return 1
    else:
        return 0  
    

In [ ]:
# Categorical 'peak hour' column 
# peak hour column for working days

def peak_hour(row):
    ph_list = [7, 8, 9, 16, 17, 18, 19]
    if all([row['holiday']==0, row['workingday']==1, row['hour'] in ph_list]):
        return 1
    else:
        return 0  
    

In [ ]:
# Categorical 'atemp'  column

# group temps in 5deg intervals (e.g. 1: 0=<atemp<4.99, 2: 5=<atemp<10.99, ... 9: atemp=>40)

def cat_atemp(atemp):

    if 0<= atemp <14.99:
        return 1
    elif 5<= atemp <29.99:
        return 2
    else:
        return 3

#     if 0<= atemp <4.99:
#         return 1
#     elif 5<= atemp <9.99:
#         return 2
#     elif 10<= atemp <14.99:
#         return 3
#     elif 15<= atemp <19.99:
#         return 4
#     elif 20<= atemp <24.99:
#         return 5
#     elif 25<= atemp <29.99:
#         return 6
#     elif 30<= atemp <34.99:
#         return 7
#     elif 35<= atemp <39.99:
#         return 8
#     else:
#         return 9
    

In [ ]:
# Categorical 'ph2' column
# Weekend peak hours (9am to 9pm )
# Holiday rentals are not significant (very much below mean daily rentals)

def peak_hour2(row):
    ph2_list = range(9,22)
    weekend = [5,6]
    if all([row['day of week']in weekend, row['hour'] in ph2_list]):
        return 1
    else:
        return 0  
    

In [ ]:
data=train.copy()


# convert ['datetime'] to datetime obj
data['datetime'] = data.apply(lambda eachrow: dt.strptime(eachrow['datetime'], "%Y-%m-%d %H:%M:%S"), axis=1)


# add 'year', 'month', 'day', 'hour' columns
data['year'] = data['datetime'].dt.year
data['month'] = data['datetime'].dt.month
data['day'] = data['datetime'].dt.day
data['hour'] = data['datetime'].dt.hour


# add 'day of week' column
data['day of week'] = data['datetime'].dt.dayofweek


# adding new features
data['am morning'] = data.apply(lambda row: ampm(row), axis=1)
data['peak hour'] = data.apply(lambda row: peak_hour(row), axis=1)
data['peak hour2'] = data.apply(lambda row: peak_hour2(row), axis=1)
data['cat atemp'] = data.apply(lambda eachrow: cat_atemp(eachrow['atemp']), axis=1)



In [ ]:
data.sample(10)

In [ ]:
#new grouped dataset
grouped1 = data.groupby(['weather']).sum()
grouped2 = data.groupby(['season']).sum() 
grouped3 = data.groupby(['hour']).sum()

#drop columns that have no meaning when summed
grouped1.drop(['holiday','workingday','temp','atemp','humidity','windspeed','season'],axis=1,inplace=True)
grouped2.drop(['holiday','workingday','temp','atemp','humidity','windspeed','weather'],axis=1,inplace=True)
grouped3.drop(['holiday','workingday','temp','atemp','humidity','windspeed','weather'],axis=1,inplace=True)
display(grouped1)
display(grouped2)
display(grouped3)

# print(grouped.index, '\n')
# grouped.loc[(1,4)]

# It appears that rentals are more affected by weather than by season. 
# Casual users drop significantly in poor weather 
# Registered rentals remain mostly consistent across seasons but drops when it is (1), spring. 

In [ ]:
group_hour_workday1 = data.loc[data['workingday']==1].groupby(['hour']).sum()
group_hour_workday0 = data.loc[data['workingday']==0].groupby(['hour']).sum()

fig, (ax1,ax2,ax3,ax4) = plt.subplots(4,1, figsize=(10,20), sharex=False)

ax1 = sns.barplot(x=group_hour_workday1.index, y='casual', data=group_hour_workday1, color='blue', ax=ax1)
ax1.set_xlabel(None)
ax2 = sns.barplot(x=group_hour_workday1.index, y='registered', data=group_hour_workday1, color='red', ax=ax2)
ax2.set_xlabel(None)
ax3 = sns.barplot(x=group_hour_workday0.index, y='casual', data=group_hour_workday0, color='mediumblue', ax=ax3)
ax3.set_xlabel(None)
ax4 = sns.barplot(x=group_hour_workday0.index, y='registered', data=group_hour_workday0, color='darkred', ax=ax4)


In [ ]:
# Correlation Heatmap without categorical columns except for binary types

data_corr = data.drop(columns=['datetime','weather','temp','count','year','day','hour','day of week'])
corrmat = data_corr.corr()
fig, ax = plt.subplots(figsize=(12,12))

sns.heatmap(corrmat, annot=True,fmt='.2f', annot_kws={'size':10},square=True)


as expected, there is presence of multicollinearity between the variables temp and atemp ('feels like temp') and we will drop the predictor temp there is high correlation between registered and count because registered users comprise a significant percentage of total rentals. however, we do not drop the predictor at this stage as we would like to understand what drives casual vs registered usage

From the heatmap, the top 3 variables most correlated with the number of rental is atemp, season, windspeed. All other variable have low correlation with count <0.05

In [ ]:
data.sample(5)

In [ ]:
# Data Splitting --- for independent 'registered' and 'casual' modelling

cols_to_drop = ['datetime','temp','registered','casual','count','cat atemp','am morning']

y_data_reg = data['registered']
X_data_reg = data.drop(cols_to_drop, axis=1)
X_train_reg,X_test_reg,y_train_reg,y_test_reg = train_test_split(X_data_reg,y_data_reg, 
                                                                 test_size=0.1)
                        
y_data_cas = data['casual']
X_data_cas = data.drop(cols_to_drop, axis=1)                    
X_train_cas,X_test_cas,y_train_cas,y_test_cas = train_test_split(X_data_cas,y_data_cas,
                                                                 test_size=0.1)                        

In [ ]:
full = [0,1,2,3,7,8,9,10,11,12,13]  
part = [0,1,2,3,12,13]            # no month,day,hour,dayofweek,cat_atemp
cat_features_reg = None
cat_features_cas = None


X_train_reg_pool = Pool(X_train_reg,
                        y_train_reg, 
                        cat_features=cat_features_reg)
X_test_reg_pool = Pool(X_test_reg,
                       y_test_reg,
                       cat_features=cat_features_reg) 


X_train_cas_pool = Pool(X_train_cas,
                        y_train_cas,
                        cat_features=cat_features_cas)
X_test_cas_pool = Pool(X_test_cas,
                       y_test_cas,
                       cat_features=cat_features_cas) 


In [ ]:
param = {
        'n_estimators':500,
        'depth':3, 
        'learning_rate':1, 
        'loss_function':'RMSE'
        }

model1 = CatBoostRegressor(n_estimators=3000,depth=8,learning_rate=0.1,loss_function='RMSE',l2_leaf_reg=20)
model2 = CatBoostRegressor(n_estimators=3000,depth=8,learning_rate=0.1,loss_function='RMSE',l2_leaf_reg=20)

model1.fit(X_train_reg_pool)
model2.fit(X_train_cas_pool)

In [ ]:
def pred_scores(data,label,model):
    
    pred = [max(0,round(num)) for num in model.predict(data)]
    
    return  {
            "RMSE": np.sqrt(metrics.mean_squared_error(label,pred)), 
            "R-Squared": metrics.r2_score(label,pred),
#             "Predictions": pred[50:200]
            }

print("TRAIN-registered:", pred_scores(X_train_reg_pool, y_train_reg, model1))
print("TRAIN-casual:", pred_scores(X_train_cas_pool, y_train_cas, model2))
print("TEST-registered:", pred_scores(X_test_reg_pool, y_test_reg, model1))
print("TEST-casual:", pred_scores(X_test_cas_pool, y_test_cas, model2))

In [ ]:
# test.csv data setup

tdata=test.copy()

tdata['datetime'] = tdata.apply(lambda eachrow: dt.strptime(eachrow['datetime'], "%Y-%m-%d %H:%M:%S"), axis=1)

tdata['year'] = tdata['datetime'].dt.year
tdata['month'] = tdata['datetime'].dt.month
tdata['day'] = tdata['datetime'].dt.day
tdata['hour'] = tdata['datetime'].dt.hour

tdata['day of week'] = tdata['datetime'].dt.dayofweek

tdata['am morning'] = tdata.apply(lambda row: ampm(row), axis=1)
tdata['peak hour'] = tdata.apply(lambda row: peak_hour(row), axis=1)
tdata['peak hour2'] = data.apply(lambda row: peak_hour2(row), axis=1)
tdata['cat atemp'] = tdata.apply(lambda eachrow: cat_atemp(eachrow['atemp']), axis=1)

test_data = tdata.drop(columns=['datetime','temp','cat atemp'])


In [ ]:
# test data to model (Pool)

test_reg_pool = Pool(test_data,
                        cat_features=cat_features_reg) 

test_cas_pool = Pool(test_data,
                        cat_features=cat_features_cas)

In [ ]:
# Test predictions

pred_reg = [max(0,round(num)) for num in model1.predict(test_reg_pool)]
pred_cas = [max(0,round(num)) for num in model2.predict(test_cas_pool)]

print(pred_reg[50:100])
print(pred_cas[50:100])

pred_total = [pred_reg[i] + pred_cas[i] for i in range(len(pred_reg))]
print(pred_total[50:100])

In [ ]:
output = pd.DataFrame(test['datetime'], columns=['datetime', 'count'])
output['count'] = pred_total
output.to_csv('submission.csv', index=False)

output